In [41]:
import pandas as pd
import os
import datetime
import time
import requests
from bs4 import BeautifulSoup
from PIL import Image
import io

class FootballDataScraper:
    def __init__(self, config_file):
        self.config_file = config_file
        self.headers = {'User-Agent': 'Mozilla/5.0'}
        self.championnats = self.load_configs()
        self.data_folder = "Database"
        self.logos_folder = "Logos"
        self.last_request_time = None
        self.request_interval = 3  # secondes
        self.max_seasons = 6

    def load_configs(self):
        # loading config of championship containing which championship are used and their urls
        return pd.read_csv(self.config_file)

    def scrape_or_update(self):
        all_data = {}
        for _, config in self.championnats.iterrows():
            championnat = config['Championnat']
            url = config['URL']
            csv_file_path = os.path.join(self.data_folder, f"{championnat}_data.csv")

            if os.path.exists(csv_file_path):
                data = pd.read_csv(csv_file_path)
                last_update = pd.to_datetime((data['Date'] + ' ' + data['Time']).max())
                if datetime.datetime.now() > last_update + datetime.timedelta(hours = 2):
                    updated_data = self.update_data(championnat, url, data)
                    all_data[championnat] = updated_data
            else:
                data = self.scrape_data(url)
                all_data[championnat] = data

            self.save_data(all_data[championnat], csv_file_path)
        return all_data

    def scrape_data(self, url):
        
        all_seasons_data = []

        for saison in range(self.max_seasons):

            self.rate_limit()

            response = requests.get(url, headers=self.headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            teams_urls = ["https://fbref.com" + equipe.get("href") 
                        for equipe in soup.select("table.stats_table")[0].find_all("a") 
                        if "squads" in equipe.get("href", "")]

            url = f"https://fbref.com{soup.find('a', class_='button2 prev').get('href')}"

            for team_url in teams_urls:

                self.rate_limit()
                team_response = requests.get(team_url, headers=self.headers)
                try:
                    team_data = pd.read_html(team_response.text, match="Scores")[0]
                except ValueError as e:
                    # Gestion de l'exception si aucune table n'est trouvée
                    print(f"Aucune table trouvée dans {team_url} - Erreur: {e}")
                    continue  # Passe à l'itération suivante de la boucle

                team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
                team_data["Team"] = team_name

                self.scrape_and_save_logo(team_url, team_name)

                # Récupération des URLs pour les statistiques détaillées
                url_stats = {
                    f"https://fbref.com{a.get('href')}" 
                    for a in BeautifulSoup(team_response.text, 'html.parser').find_all("a") 
                    if "matchlogs/all_comps" in a.get('href', '') and 
                    any(substring in a.get('href', '') for substring in ["passing/", "shooting", "possession/", "defense/", "keeper"])
                }

                # Traitement des statistiques détaillées
                for stats_url in url_stats:

                    self.rate_limit()

                    stats_response = requests.get(stats_url, headers=self.headers)

                    try:
                        detailed_stats = pd.read_html(stats_response.text)[0]
                    except ValueError as e:
                        print(f"Aucune table trouvée dans {stats_url} - Erreur: {e}")
                        continue  # Passe à l'itération suivante de la boucle

                    # Nettoyage des colonnes du DataFrame
                    if detailed_stats.columns.nlevels > 1:
                        detailed_stats.columns = [f"{col}_{branch}" 
                                                if "For" not in col and "Unnamed:" not in col 
                                                else f"{branch}" 
                                                for col, branch in detailed_stats.columns]

                    columns_to_drop = ["Time", "Comp", "Round", "Day", "Venue", "Result", "GF", "GA", "Opponent", "Poss"] + [col for col in detailed_stats.columns if 'Report' in col]
                    columns_to_drop = [col for col in columns_to_drop if col in detailed_stats.columns]

                    detailed_stats.drop(columns_to_drop, axis=1, inplace=True)

                    team_data = team_data.merge(detailed_stats, on="Date")

                # Ajout des données de l'équipe au résultat global
                all_seasons_data.append(team_data)

        return pd.concat(all_seasons_data, ignore_index=True)

    def update_data(self, championnat, url, existing_data):

        latest_data = []
        futur_matchweek = []

        response = requests.get(url, headers=self.headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        teams_urls = ["https://fbref.com" + equipe.get("href") 
                    for equipe in soup.select("table.stats_table")[0].find_all("a") 
                    if "squads" in equipe.get("href", "")]

        # Traitement similaire à scrape_ligue1_data() pour chaque équipe
        for team_url in teams_urls:
            rate_limit()
            team_response = requests.get(team_url, headers=headers)
            try:
                team_data = pd.read_html(team_response.text, match="Scores")[0]
            except ValueError as e:
                # Gestion de l'exception si aucune table n'est trouvée
                print(f"Aucune table trouvée dans {team_url} - Erreur: {e}")
                continue  # Passe à l'itération suivante de la boucle

            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
            team_data["Team"] = team_name

            futur_matchweek.append(team_data)

            # Récupération des URLs pour les statistiques détaillées
            url_stats = {
                f"https://fbref.com{a.get('href')}" 
                for a in BeautifulSoup(team_response.text, 'html.parser').find_all("a") 
                if "matchlogs/all_comps" in a.get('href', '') and 
                any(substring in a.get('href', '') for substring in ["passing/", "shooting", "possession/", "defense/", "keeper"])
            }

            for stats_url in url_stats:

                self.rate_limit()

                stats_response = requests.get(stats_url, headers=self.headers)
                
                try:
                    detailed_stats = pd.read_html(stats_response.text)[0]
                except ValueError as e:
                    print(f"Aucune table trouvée dans {stats_url} - Erreur: {e}")
                    continue  # Passe à l'itération suivante de la boucle


                # Nettoyage des colonnes du DataFrame
                if detailed_stats.columns.nlevels > 1:
                    detailed_stats.columns = [f"{col}_{branch}" 
                                            if "For" not in col and "Unnamed:" not in col 
                                            else f"{branch}" 
                                            for col, branch in detailed_stats.columns]

                columns_to_drop = ["Time", "Comp", "Round", "Day", "Venue", "Result", "GF", "GA", "Opponent", "Poss"] + [col for col in detailed_stats.columns if 'Report' in col]
                columns_to_drop = [col for col in columns_to_drop if col in detailed_stats.columns]

                detailed_stats.drop(columns_to_drop, axis=1, inplace=True)

                team_data = team_data.merge(detailed_stats, on="Date")

            # Ajout des données de l'équipe au résultat global
            all_seasons_data.append(team_data)

        # Retourne les données concaténées de toutes les équipes
        return pd.concat(latest_data, ignore_index=True), pd.concat(futur_matchweek, ignore_index=True)

    def save_data(self, data, file_path):
        # Enregistrer les données dans un fichier CSV
        data.to_csv(file_path, index=False)

    def rate_limit(self):
        if self.last_request_time is not None:
            elapsed_time = time.time() - self.last_request_time
            if elapsed_time < self.request_interval:
                time.sleep(self.request_interval - elapsed_time)
        self.last_request_time = time.time()

    def access_data(self, championnat):
        csv_file_path = os.path.join(self.data_folder, f"{championnat}_data.csv")
        
        if os.path.exists(csv_file_path):
            data = pd.read_csv(csv_file_path)
            return data
        else:
            print(f"Les données pour le championnat '{championnat}' n'existent pas.")

    def scrape_and_save_logo(self, page_url, file_name):

        file_path = os.path.join(self.logos_folder, file_name + '.png')
        if os.path.exists(file_path):
            print(f"Le logo existe déjà à {file_path}")
            return

        try:
            self.rate_limit()
            response = requests.get(page_url, headers=self.headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            logo_img = soup.find('img', {'class': 'teamlogo'})

            if logo_img and logo_img.get('src'):
                logo_url = logo_img['src']
                self.rate_limit()
                img_response = requests.get(logo_url)

                if img_response.status_code == 200:
                    # Convertir l'image en PNG
                    image = Image.open(io.BytesIO(img_response.content))
                    file_path = os.path.join(self.logos_folder, file_name)
                    if not file_path.endswith('.png'):
                        file_path += '.png'
                    image.save(file_path, 'PNG')
                    print(f"Logo sauvegardé en PNG à {file_path}")
                else:
                    print("Erreur lors du téléchargement de l'image")
            else:
                print("Logo non trouvé sur la page")

        except Exception as e:
            print(f"Une erreur s'est produite : {e}")

            


    









In [42]:
allo = FootballDataScraper("config.csv")
allo.scrape_or_update()

/tmp/ipykernel_23442/3818153079.py:67: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team_data = pd.read_html(team_response.text, match="Scores")[0]


Le logo existe déjà à Logos/Girona.png


/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.


Le logo existe déjà à Logos/Real Madrid.png


/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.


...

Logo sauvegardé en PNG à Logos/De Graafschap.png


/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.


Logo sauvegardé en PNG à Logos/NAC Breda.png


/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/tmp/ipykernel_23442/3818153079.py:94: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.


{'La Liga':             Date   Time     Comp         Round  Day Venue Result   GF   GA  \
 0     2023-08-12  17:00  La Liga   Matchweek 1  Sat  Away      D  1.0  1.0   
 1     2023-08-20  19:00  La Liga   Matchweek 2  Sun  Home      W  3.0  0.0   
 2     2023-08-26  21:30  La Liga   Matchweek 3  Sat  Away      W  2.0  1.0   
 3     2023-09-03  14:00  La Liga   Matchweek 4  Sun  Home      W  1.0  0.0   
 4     2023-09-18  21:00  La Liga   Matchweek 5  Mon  Away      W  4.0  2.0   
 ...          ...    ...      ...           ...  ...   ...    ...  ...  ...   
 5075  2019-04-25  19:30  La Liga  Matchweek 34  Thu  Away      L    0    5   
 5076  2019-04-28  20:45  La Liga  Matchweek 35  Sun  Home      W    1    0   
 5077  2019-05-04  13:00  La Liga  Matchweek 36  Sat  Away      L    1    4   
 5078  2019-05-12  18:30  La Liga  Matchweek 37  Sun  Home      L    1    2   
 5079  2019-05-18  20:45  La Liga  Matchweek 38  Sat  Away      D    2    2   
 
            Opponent  ...  Challenges_A